# Kubeflow Metrics Training

In [1]:
import kfp
from kfp import compiler
import kfp.components as comp
import kfp.dsl as dsl

EXPERIMENT_NAME = 'metrics-training'

In [2]:
def train():
    
    # it is not recommended to inlucde imports in the function
    # just used for simplicity of the example
    import tensorflow as tf
    from tensorflow.python.lib.io import file_io
    import json
    
    mnist = tf.keras.datasets.mnist
    
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5)

    loss, accuracy = model.evaluate(x_test, y_test)
    print(model.metrics_names)
    
    metrics = {
        'metrics': [{
          'name': 'accuracy-score', # The name of the metric. Visualized as the column name in the runs table.
          'numberValue':  accuracy, # The value of the metric. Must be a numeric value.
          'format': "PERCENTAGE",   # The optional format of the metric. Supported values are "RAW" (displayed in raw format) and "PERCENTAGE" (displayed in percentage format).
        }]
      }
    with file_io.FileIO('/mlpipeline-metrics.json', 'w') as f:
        json.dump(metrics, f)

    return 'done'

In [3]:
train_operation = comp.func_to_container_op(train)

In [4]:
@dsl.pipeline(
   name='keras train to show metrics',
   description='A pipeline thats used to show the usage of metrics.'
)

def metrics_pipeline():
    train_task = train_operation()  

In [5]:
pipeline_func = metrics_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [6]:
arguments = {}

import kfp
client = kfp.Client()
#experiment = client.create_experiment(EXPERIMENT_NAME)
experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)

run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

/opt/conda/lib/python3.6/site-packages/kfp/_client.py:157: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)
